In [1]:
import selenium

from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import pymysql

db = pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd='1234', db='melon', charset='utf8')

cursor = db.cursor()
    
for year in range(2012, 2022):
    link = "https://www.melon.com/chart/age/index.htm?chartType=YE&chartGenre=KPOP&chartDate={}#params%5Bidx%5D=1".format(year)

    driver = webdriver.Chrome()
    driver.get(link)

    html = driver.page_source

    time.sleep(3)
    driver.close()
    driver.quit()
    bs = BeautifulSoup(html, 'lxml')

    idx = len(bs.find_all('tr', {'class':'lst50'}))
    idx2 = len(bs.find_all('tr', {'class':'lst100'}))
    tr_all = bs.find_all('tr', {'class':'lst50'})
    tr_all2 = bs.find_all('tr', {'class':'lst100'})

    rank = 1
    
    for i in range(0, idx):
        song_name = tr_all[i].find('div', {'class':'ellipsis rank01'}).text.strip()
        singer_name = tr_all[i].find('div',{'class':'ellipsis rank02'}).find('a').text.strip()
        likes = tr_all[i].find('span', {'class':'cnt'}).text.replace('\n', ' ').replace('총건수 ', '')
        sql = """INSERT INTO top100 VALUES(
        {0}, {1}, "{2}", "{3}", "{4}");""".format(year, rank, song_name, singer_name, likes)
    
        rank += 1
        cursor.execute(sql)
    
    for i in range(0, idx2):
        song_name = tr_all2[i].find('div', {'class':'ellipsis rank01'}).text.strip()
        singer_name = tr_all2[i].find('div',{'class':'ellipsis rank02'}).find('a').text.strip()
        likes = tr_all2[i].find('span', {'class':'cnt'}).text.replace('\n', ' ').replace('총건수 ', '')
        sql = """INSERT INTO top100 VALUES(
        {0}, {1}, "{2}", "{3}", "{4}");""".format(year, rank, song_name, singer_name, likes)
    
        rank += 1
        cursor.execute(sql)

db.commit()
db.close()